In [2]:
import os
os.chdir('./..')

In [3]:
import data
import pandas as pd
from tqdm import tqdm

In [4]:
def _session_score_negative_value_seen_elem(df):
    # initialize the dict that will contain the scores of the references
    score = {}
    ref_last_clickout = None
    clickout_items_not_final = None
    session_impressions = None
    numeric_references = []
    
    # get all the impressions of the clickouts
    clickouts = df[df['action_type']=='clickout item']
    
    #check if there is at least one clickout
    if len(clickouts) > 0:
        ref_last_click = int(clickouts.tail(1)['reference'])
        if ref_last_click != -1:
            ref_last_clickout = ref_last_click
        
        # check if in the session there are more than 1 clickout
        if len(clickouts) > 1:
        # remove the row of last clickout
            clicked_ref = list(map(int,(clickouts['reference'].unique())))
            # remove the last click from the clicked one
            if ref_last_clickout is not None:
                clicked_ref.remove(ref_last_clickout)
            clickout_items_not_final = clicked_ref
        
        session_impressions = list(map(int,clickouts[clickouts['reference']!=-1]['reference'].unique()))
        # remove from the impressions the last clicked
        if ref_last_clickout is not None:
            session_impressions.remove(ref_last_clickout)
    
    # take all the numeric reference in the session
    references = list(df['reference'].unique())
    for r in references:
        try:
            r = int(r)
            if r != -1:
                numeric_references.append(r)
        except ValueError:
            continue
    
    last_click_score = 1 
    clicked_ref_score = 0.5
    impr_not_seen_score = -0.5
    seen_ref_score = -1
    
    # if there isn't a clickout in the session we assume all reference are not pleased the user
    if ref_last_clickout is None:
        for ref in numeric_references:
            score[ref] = impr_not_seen_score
    
    # if there is 1 clickout 
    if ref_last_clickout is not None:
        
        score[ref_last_clickout] = last_click_score
        for ref in numeric_references:
            score[ref] = seen_ref_score
        for impr in session_impressions:
            if impr not in score:
                score[impr] = impr_not_seen_score        
        # if there are more than 1 clickout
        if clickout_items_not_final is not None:
            for click in clickout_items_not_final:
                score[click] = clicked_ref_score
    return score
    
    

In [10]:
train_df = data.train_df(mode='small')
test_df = data.test_df(mode='small')

test_df.fillna({'reference': -1}, inplace=True)
train_df.fillna({'reference': -1}, inplace=True)

df = pd.concat([train_df, test_df])[['session_id', 'user_id', 'action_type', 'reference', 'impressions']]

session_groups = df.groupby('user_id')

tqdm.pandas()
sessions_score = session_groups.progress_apply(_session_score_negative_value_seen_elem).values

100%|██████████| 56134/56134 [01:07<00:00, 832.67it/s]


In [13]:
#print(sessions_score)
print(sessions_score.keys())

Index(['0004IOZI7CKF', '0008B0X0HC39', '000EKQEFMW6S', '000ZN9219G41',
       '001OCXQ9PXET', '001TEVEVUEBE', '0020N6R8BEDO', '002BISXP1U1Q',
       '002F746OAPLZ', '002IXCPKK20W',
       ...
       'ZZQ231GOCXQB', 'ZZRDO4E49IX8', 'ZZT1WQYKDMTF', 'ZZTZ7W649SCD',
       'ZZW47LRIK2MI', 'ZZW8VOP35SN0', 'ZZWHEYWBUNJB', 'ZZWZBXI100ZW',
       'ZZYRG6A6KZB7', 'ZZZ4CA1PZ1VU'],
      dtype='object', name='user_id', length=56134)
